In [498]:
import torch
import numpy as np
import math
from scipy import integrate
#from common.ffn.ffn_relu import ParametricReLUNet
from common.ffn.ffn_tanh import TanhNet
from common.ffn.ffn_base import FFNGmetricLogging

***
##### Theoretical values
***
Consider the FFN with input data dimension ${n_0}$, all layers have width ${n}$; the width at l-layer is denoted by ${n_l}$. For preactivation on l-layer $z^{(l)}$ activation function is $σ(z^{(l)})$, or $σ^{(l)}$. Preactivation weights are initialized with a centered normal distribution with variance $1/n$, except in the first layer, where the variance is $1/{n_0}$; bias is constantly zero. As mentioned, the preactivation at layer l for the trainset with points $α_1...α_N∈D$ is $z^{(l)}$. Consider the distribution:

$$p(z^{(l)})_{g,v}=\frac{1}{Z_{g,v}}exp(-\frac{1}{2}\sum \limits _{k=1} ^{n_l}\sum \limits _{α_1,α_2∈D}g^{α_1,α_2}_{(l)}z^{(l)}_{k,α_1}z^{(l)}_{k,α_2})(1+\frac{1}{8}\sum \limits _{k_1,k_2=1} ^{n^l}\sum \limits _{α_1,α_2,α_3,α_4∈D}v^{(α_1,α_2)(α_3,α_4)}_{(l)}z^{(l)}_{k_1,α_1}z^{(l)}_{k_1,α_2}z^{(l)}_{k_2,α_3}z^{(l)}_{k_2,α_4}) (1)$$

Here, $g^{α_1,α_2}_{(l)}$ and $v^{(α_1,α_2)(α_3,α_4)}_{(l)}$ are calculated via $G^{(l)}_{α_1,α_2}$ and $v^{(l)}_{(α_1,α_2)(α_3,α_4)}$:

$$G^{(l+1)}_{α_1,α_2}=<σ^{(l)}_{α_1}σ^{(l)}_{α_2}>_{g^{(l)}}+\frac{1}{8}\sum \limits _{β_1,β_2,β_3,β_4∈D}v^{(β_1,β_2)(β_3,β_4)}_{(l)}(<σ^{(l)}_{α_1}σ^{(l)}_{α_2}z^{(l)}_{β_1,β_2}(z^{(l)}_{β_3,β_4}+2ng^{(l)}_{β_3,β_4})>_{g^{(l)}}-2<σ^{(l)}_{α_1}σ^{(l)}_{α_2}>_{g^{(l)}}g^{(l)}_{β_1,β_3}g^{(l)}_{β_2,β_4}) (2)$$

Formula (2) also matches (4.61) in [1]. In this formula $σ(z^{(l)})=σ^{(l)}$; $<⋅>_{g^{(l)}}$ means gaussian integral with covariance matrix $g^{(l)}$ for all greek letters variables mentioned inside <⋅>; $z^{(l)}_{β_1,β_2}=z^{(l)}_{β_1}z^{(l)}_{β_2}-g^{(l)}_{β_1,β_2}$

$$v^{(l+1)}_{(α_1,α_2)(α_3,α_4)}=\frac{1}{n}(<σ^{(l)}_{α_1}σ^{(l)}_{α_2}σ^{(l)}_{α_3}σ^{(l)}_{α_4}>_{g^{(l)}}-<σ^{(l)}_{α_1}σ^{(l)}_{α_2}>_{g^{(l)}}<σ^{(l)}_{α_3}σ^{(l)}_{α_4}>_{g^{(l)}})+\frac{1}{4}\sum \limits _{β_1,β_2,β_3,β_4∈D}v^{(β_1,β_2)(β_3,β_4)}_{(l)}<σ^{(l)}_{α_1}σ^{(l)}_{α_2}z^{(l)}_{β_1,β_2}>_{g^{(l)}}<σ^{(l)}_{α_3}σ^{(l)}_{α_4}z^{(l)}_{β_3,β_4}>_{g^{(l)}} (3)$$

Formula (3) matches (4.90) in [1] if the subleading $1/n^2$ correction is neglected and the following sumstitutions are applied: $g^{α_1,α_2}=G^{α_1,α_2}+O(1/n), v^{(β_1,β_2)(β_3,β_4)}=\frac{1}{n}V^{(β_1,β_2)(β_3,β_4)}+O(1/n^2), v_{(β_1,β_2)(β_3,β_4)}=??$
Based on previous values, $g^{α_1,α_2}_{(l+1)}$ and $v^{(α_1,α_2)(α_3,α_4)}_{(l+1)}$ can be computed:

$$g^{α_1,α_2}_{(l+1)}=G^{α_1,α_2}_{(l+1)}+\sum \limits _{β_1,β_2,β_3,β_4∈D}v^{(l+1)}_{(β_1,β_2)(β_3,β_4)}G^{α_1,β_1}_{(l+1)}(G^{β_2,β_3}_{(l+1)}G^{β_4,α_2}_{(l+1)}+\frac{n}{2}G^{β_2,α_2}_{(l+1)}G^{β_3,β_4}_{(l+1)}) (4)$$
$$v^{(α_1,α_2)(α_3,α_4)}_{(l+1)}=\sum \limits _{β_1,β_2,β_3,β_4∈D}G^{α_1,β_1}_{(l+1)}G^{α_2,β_2}_{(l+1)}G^{α_3,β_3}_{(l+1)}G^{α_4,β_4}_{(l+1)}v^{(l+1)}_{(β_1,β_2)(β_3,β_4)} (5)$$

The theorem is as follows: Suppose $q(z^{(l)})$ is a true preactivation distribution at l-layer in FFN, $σ(x)=O(x^k), k∈N$. Then, for any S∈Ω, 

$$|\int_{z^{(l)}∈S}q(z^{(l)})dz-\int_{z^{(l)}∈S}p(z^{(l)})_{g,v}dz^{(l)}|=O(\frac{1}{n^{1.49}}) (6)$$

***
##### Case 1: theoretical values for 1-point train set; all α and β are equal to 1. Input width is 3, all layer widths are 100; number of layers is 5. Activation function is hyperbolic tangent.
***
Creating FFN, initialising trainset and variance coefficients:

In [486]:
'''n0: # dimension of x
    nk: # hidden nodes
    nl: # dimension of y
    l: # number of layers
    nd: # number of points in train-set'''
n0,nk,nl,l=5,100,100,5
nd=4

testNet = TanhNet(n0=n0,nk=nk,nl=nl,l=l)
testNet.set_log_level("info")
xx = np.random.normal(size=(n0, nd)).astype(np.float32)
cb, cw = 0, 1


FeedForwardNet created with n0=5, nk=100, nl=100, l=5, bias_on=False


In [487]:
'''
def gg_to_pos(g1, g2):
    return int((g1-1)*g1/2+g2)-1

def vv_to_pos(v1, v2, v3, v4):
    aa = gg_to_pos(v1, v2)
    return int((aa+1)*aa/2 + gg_to_pos(v3, v4))
'''

'''Array to save g-coeffs for all l-layers: number of g-coeffs per layer x number of layers
    number of g-coeffs per layer - number of combinations from nd by 2 with repetition = [(nd+1)nd]/2.
    In g^{α_1,α_2} α_1>=α_2
'''
matrix_gg_top = np.zeros((l, nd,nd))##gg_to_pos(nd,nd)+1))#int(((nd+1)*nd)/2)))
matrix_gg_bottom = np.zeros_like(matrix_gg_top)
'''Array to save v-coeffs for all l-layers: number of v-coeffs per layer x number of layers
    number of v-coeffs per layer - [(N+1)N]/2=nd(nd+1)(nd(nd+1)+2)/8.
    In v^{(α_1,α_2),(α_3,α_4)} α_1>=α_2, α_3>=α_4, α_1+α_2>=α_3+α_4
'''
matrix_vv_top = np.zeros((l, nd,nd,nd,nd)) #vv_to_pos(nd,nd,nd,nd)+1))#int(nd*(nd+1)*(nd*(nd+1)+2)/8)))
matrix_vv_bottom = np.zeros_like(matrix_vv_top)


In [488]:
'''
test = ['']*(gg_to_pos(5, 5)+1)
for ii in range(1, 6):
    for jj in range(1, ii+1):
        #print(str(ii) +'-'+ str(jj)+':'+str(gg_to_pos(ii, jj)))
        test[gg_to_pos(ii, jj)] = str(ii) +'-'+ str(jj)

print(test)

top=5
test = ['']*(vv_to_pos(top, top, top, top)+1)
#cnt=0
for i1 in range(1, top+1):
    for i2 in range(1, i1+1):
        for j1 in range(1, i1+1):
            for j2 in range(1, min(i1+i2-j1, j1)+1):
                test[vv_to_pos(i1, i2, j1, j2)] = str(i1) +'-'+ str(i2) +'-'+ str(j1) +'-'+ str(j2)
                #cnt+=1
                #print(str(i1) +'-'+ str(i2) +'-'+ str(j1) +'-'+ str(j2))

print(len(test))
'''


"\ntest = ['']*(gg_to_pos(5, 5)+1)\nfor ii in range(1, 6):\n    for jj in range(1, ii+1):\n        #print(str(ii) +'-'+ str(jj)+':'+str(gg_to_pos(ii, jj)))\n        test[gg_to_pos(ii, jj)] = str(ii) +'-'+ str(jj)\n\nprint(test)\n\ntop=5\ntest = ['']*(vv_to_pos(top, top, top, top)+1)\n#cnt=0\nfor i1 in range(1, top+1):\n    for i2 in range(1, i1+1):\n        for j1 in range(1, i1+1):\n            for j2 in range(1, min(i1+i2-j1, j1)+1):\n                test[vv_to_pos(i1, i2, j1, j2)] = str(i1) +'-'+ str(i2) +'-'+ str(j1) +'-'+ str(j2)\n                #cnt+=1\n                #print(str(i1) +'-'+ str(i2) +'-'+ str(j1) +'-'+ str(j2))\n\nprint(len(test))\n"

Preactivation on 1st layer $z^{(1)}$ has Gaussian distribution as a sum of gaussians weights for each neuron:
$$z^{(1)}_{k,α}=b^{(1)}_k+\sum \limits _{s=1} ^{n_0}w^{(1)}_{k,s}z^{(0)}_{s,α} (7)$$
$$E(z^{(1)}_{k,α})=0 (8)$$
$$E(z^{(1)}_{k,α}z^{(1)}_{k,β})=\frac{1}{n^0}\sum \limits _{s=1} ^{n_0}z^{(0)}_{s,α}z^{(0)}_{s,β}=G^{(1)}_{α,β} (9)$$
When α=β=1,
$$E(z^{(1)}_{k,α})^2=\frac{1}{n_0}\sum \limits _{s=1} ^{n_0}(z^{(0)}_{s,α})^2=G^{(1)}_{α,α} (10)$$
Value $v^{(1)}=0$, $G^{(1)}=(G_{(1)})^{-1}, g^{α_1,α_2}_{(1)}=G^{α_1,α_2}_{(1)}$

In [489]:
#matrix_gg_bottom = np.zeros((nd,nd))

for ii in range(0, nd):
    for jj in range(0, ii+1):
        value = FFNGmetricLogging.G_xx(xx[:,ii], xx[:,jj], cb, cw)
        #coeffs_gg_bottom[0, gg_to_pos(ii+1, jj+1)] = value
        matrix_gg_bottom[0, ii, jj] = value
        matrix_gg_bottom[0, jj, ii] = value

matrix_gg_top[0] = np.linalg.inv(matrix_gg_bottom[0])


In [490]:
print(np.linalg.det(matrix_gg_bottom[0]))
print(np.linalg.matrix_rank(matrix_gg_bottom[0]))
print(matrix_gg_bottom[0])
print(matrix_gg_top[0])

0.002421068877084027
4
[[ 1.25610495  0.45728536 -0.32321093  0.03477004]
 [ 0.45728536  0.24422181  0.05146189  0.15778646]
 [-0.32321093  0.05146189  0.55259795  0.25450718]
 [ 0.03477004  0.15778646  0.25450718  0.54296021]]
[[ 19.16271649 -41.74127592  12.84739888   4.88095541]
 [-41.74127592  95.99411722 -27.72159702 -12.22903761]
 [ 12.84739888 -27.72159702  10.93491545   2.10765656]
 [  4.88095541 -12.22903761   2.10765656   4.09505484]]


In [495]:
#((np.matrix(X).T*np.matrix(A)).A * Y.T.A).sum(1)
args=(1.0, 1.0, 1.0, 1.0)
print(np.dot((np.matrix(args)*np.matrix(matrix_gg_top[0])).A, args))
print(np.dot(np.dot(args, matrix_gg_top[0]), args))
np.exp(-0.5*np.dot(np.dot(args, matrix_gg_top[0]), args))
#zz = np.matrix(args)
#print(((zz*np.matrix(matrix_gg_top[0])).A * zz.T.A).sum(1))

[6.47500457]
6.4750045682206565


0.039261837647488515

Preactivation on 2nd and subsequent layers $z^{(l)}$ is count by formulas (2)-(5)

In [501]:
#for ii in range(1, nd+1):
#    for jj in range(1, ii+1):
current_layer = 0
train_index1_zerobased, train_index2_zerobased = 0,0
def density(*args):
    return math.exp(-0.5*np.dot(np.dot(args, matrix_gg_top[0]), args)) #np.e**
    #return np.exp(-0.5*np.dot((np.matrix(args)*np.matrix(matrix_gg_top[0])).A, args))

#val = integrate.nquad(density, [[-np.inf, np.inf]]*nd)
val = integrate.nquad(density, [[-10, 10]]*nd, opts=dict(epsabs=5e-02, epsrel=5e-02, limit=100))
print(val)


(1.9419379629607925, 0.0499984677530676)


In [493]:
(((2*np.pi)**nd)*np.linalg.det(matrix_gg_bottom[0]))**0.5 #1.9419379629607925

1.942510205471129